# Exercise 2

### Part 1. Implementing a merkle tree

The general algo I think is to just split data blocks into adjacent pairs, apply hash, make a new block labeled with concat of two hashes which is child blocks point to, repeat until you don't have pairs. 

In [4]:
data = [b"Four score and seven years ago", b"our fathers brought forth",
        b"on this continent", b"a new nation", b"conceived in Liberty", 
        b"and dedicated", b"to the proposition that", b"all men are created equal",
       b"that this nation", b"under God", b"shall have a new birth of freedom",
        b"and that government", b"of the people", b"by the people", b"for the people", 
        b"shall not perish from the earth"]


In [5]:
from hashlib import sha256

In [6]:
class Node():
    def __init__(self, data):
        self.data = data
        self.leftChild = None
        self.rightChild = None
        self.parent = None

In [54]:
class MerkleTree():
    def __init__(self, data):
        self.data = data
        self.root = None
        self.nodes = []
        
    def hashFunction(self, datainput):
        sha_compute = sha256()
        sha_compute.update(datainput)
        return sha_compute.digest()
    
    def createLeaves(self):
        # I am assuming even number of input blocks -- we can check for this later
        firstLevel = []
        for block in data:
            print(block)
            print( self.hashFunction(block) )
            newNode = Node(self.hashFunction(block))
            firstLevel.append(newNode)
        self.nodes.append(firstLevel)
        
    def createTree(self):
        currLevel = 0
        while len(self.nodes[currLevel]) > 1:
            self.nodes.append([])
            # get pairs of adjacent blocks (1+2, 3+4, ...)
            pairs = [x.data+y.data for (x,y) in list(zip(self.nodes[currLevel][::2],self.nodes[currLevel][1::2]))]
            count = 0
            for pair in pairs:
                newNode = Node( self.hashFunction(pair) )
                newNode.leftChild = self.nodes[currLevel][count]
                newNode.rightChild = self.nodes[currLevel][count+1]
                self.nodes[currLevel+1].append(newNode)
                self.nodes[currLevel][count].parent = self.nodes[currLevel+1][-1] 
                self.nodes[currLevel][count+1].parent = self.nodes[currLevel+1][-1]
                count += 2
            currLevel += 1
        self.root = self.nodes[currLevel][0]
        
    def printTree(self):
        pass
    
    def traverseToRoot(self, datablock):
        def findLeaf(datablock):
            for leaf in self.nodes[0]:
                if self.hashFunction(datablock) == leaf.data:
                    return leaf
            raise ValueError("data block not found" )
        currNode = findLeaf(datablock)
        chainofNodes = []
        while currNode:
            chainofNodes.append(currNode.data)
            currNode = currNode.parent
#         print(' -> '.join([str(data) for data in chainofNodes]))
        return chainofNodes
            

In [55]:
test = MerkleTree(data)
test.createLeaves()
test.createTree()
# test.createTree()

b'Four score and seven years ago'
b'!7B\xbdY\xf1\xd8\xfe\x84\x8bn\xa9FG\xddFS\x10\xb8\xd8\x16#MZ\x95-\xc6E\xfa2\x07\x07'
b'our fathers brought forth'
b'\xecQ\x85\xdbp\x19=\xee\x9a\x997\x8d\x0b\xed\xacU\xc1\xfa\x9b<^\x11\xf2\x91\xa5z\xd6\xae\xf8\xd8\xebQ'
b'on this continent'
b'\xa9\x19\xe9J<\x99\x1b\xde\x15\x9b\x8e\x95ON\xc1h\x17\xa8\xfep\xberw\xde=\x81\x11\xaf\xd6$|c'
b'a new nation'
b'%\x07/\xa6\xb4\x81\xb1#\xe7?s\xe2.l\x91\xb8W\xc3\xb8:\x93\xbb\xbaHl\x8cHZ\x18iO\xa6'
b'conceived in Liberty'
b'6\xec3~\xa9\xd2\xcbU\xb5\xde+$\x10\xb7\xfc\x85U\xd3\xf60\xbdB\t\x83\xa5\x8a<\x0c\x99Ns\xfd'
b'and dedicated'
b'7\xe9\xa8v1\x8d\x8c\xa4\xc5\x802`\xad\xe2h\xd4z\x9bN\x80\xc0}<\xe3\xc7\x8a\xe6\x95\x9c\xa2b\x88'
b'to the proposition that'
b'#\xd5\xc5\xc4|ex,\x8f6\x89m\xeec\x82`}\x08\xa0\xe4\xf1\x8b\xba~\xae\xfc_\xc4A\x81\x10@'
b'all men are created equal'
b'\xd3\x03ic$\xfb\xa4\x0f5;U\xcf\xad\x97q\x90/\xf1\xbcR2\xe3\xcf\x0c\x13\xa4+\xc0F:\x03\x84'
b'that this nation'
b'\xcc\x07\xc7\xf1\x07\x90\xc3\x

In [56]:
# print(test.root[0].data)
# print(test.root[0].leftChild.data)
# print(test.root[0].rightChild.data)
# print( test.nodes[-1][0].data )
# print([test.nodes[-2][i].data for i in range(2)])
print(test.nodes[0][0].parent.data)
print(test.nodes[1][0].data)

b'F\xbc_\xf1\xbdKD\x85\x96hck\xf0 \xc3\xeaF\x94 `\xecL\xb8\xc5\xa5k\xe0&\x8d`3E'
b'F\xbc_\xf1\xbdKD\x85\x96hck\xf0 \xc3\xeaF\x94 `\xecL\xb8\xc5\xa5k\xe0&\x8d`3E'


In [61]:
teststring = b'Four score and seven years ago'
print("checking if data string {} is in merkle tree...".format(teststring))
result = test.traverseToRoot(teststring)
print( result[-1] == test.root.data )

teststring = b'hi abe lincoln'
print("checking if data string {} is in merkle tree...".format(teststring))
result = test.traverseToRoot(teststring)
print( result[-1] == test.root.data )



checking if data string b'Four score and seven years ago' is in merkle tree...
True
checking if data string b'hi abe lincoln' is in merkle tree...


ValueError: data block not found

In [ ]:
array = ['a','b','c','d','e','f']
shit = list(zip(array[::2],array[1::2]))
[x+y for (x,y) in shit]

In [ ]:
newshit = []
newshit.append(array)
newshit.append(array)
newshit[0][0]
newshit[0][-1]
